In [1]:
import sys
import mysql.connector
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import pandas as pd
import sys
import mysql.connector
import os
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

year = datetime.date.today().year

# quincena=int(sys.argv[1])+1
quincena=22
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-16"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-15" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

from dotenv import load_dotenv
load_dotenv()
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id = 1')

In [3]:


def fix_commissions(cnx):
    cursor = cnx.cursor(dictionary=True)
    
    try:
        # A. Eliminar comisiones DGI inválidas (Vendedor principal o duplicados con compartida)
        delete_query = """
        DELETE FROM comissions 
        WHERE description = 'DGI' AND (
            seller_id IN (SELECT seller_id FROM internal_orders WHERE id = comissions.order_id)
            OR EXISTS (
                SELECT 1 FROM (SELECT * FROM comissions) AS c2 
                WHERE c2.order_id = comissions.order_id 
                AND c2.seller_id = comissions.seller_id 
                AND c2.description = 'COMPARTIDA'
            )
        )
        """
        cursor.execute(delete_query)
        print(f"Eliminadas {cursor.rowcount} comisiones DGI inválidas.")

        # B. Insertar comisiones DGI faltantes
        # Primero obtenemos todos los pedidos y todos los socios con DGI
        insert_query = """
        INSERT INTO comissions (order_id, seller_id, percentage, description)
        SELECT io.id, s.id, s.dgi, 'DGI'
        FROM internal_orders io
        JOIN sellers s ON s.dgi > 0
        WHERE s.id <> io.seller_id
        AND NOT EXISTS (
            SELECT 1 FROM comissions c 
            WHERE c.order_id = io.id AND c.seller_id = s.id
        )
        """
        cursor.execute(insert_query)
        print(f"Insertadas {cursor.rowcount} comisiones DGI faltantes.")
        
        cnx.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        cnx.rollback()
    finally:
        cursor.close()

# Llamada a la función (asumiendo que cnx ya existe)
fix_commissions(cnx)

Eliminadas 3 comisiones DGI inválidas.
Insertadas 36 comisiones DGI faltantes.


In [8]:
query="""
SELECT io.id AS order_id, s.id AS seller_id, s.dgi
FROM internal_orders io
CROSS JOIN sellers s
WHERE s.dgi > 0 
  AND s.id <> io.seller_id -- No es el vendedor principal
  AND NOT EXISTS (
      SELECT 1 FROM comissions c 
      WHERE c.order_id = io.id AND c.seller_id = s.id
  );

"""

In [2]:
print(pd.read_sql("SELECT COUNT(*) FROM internal_orders", cnx))

   COUNT(*)
0        53


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
 cursor = cnx.cursor(dictionary=True)

InternalError: Unread result found

In [13]:
cursor.execute(query)

In [4]:
6153 +1512+3903+226

11794